In [4]:
import pymysql
import requests

### 下載分析pm2.5 opendata資料 ###

In [5]:
url="https://data.moenv.gov.tw/api/v2/aqx_p_02?api_key=540e2ca4-41e1-4186-8497-fdd67024ac44&limit=1000&sort=datacreationdate%20desc&format=JSON"
url

'https://data.moenv.gov.tw/api/v2/aqx_p_02?api_key=540e2ca4-41e1-4186-8497-fdd67024ac44&limit=1000&sort=datacreationdate%20desc&format=JSON'

In [6]:
resp=requests.get(url,verify=False)
resp

c:\Users\USER\Desktop\django\pm25_MYSQL\.venv\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'data.moenv.gov.tw'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


<Response [200]>

In [7]:
datas=resp.json()['records']
datas

[{'site': '員林',
  'county': '彰化縣',
  'pm25': '5',
  'datacreationdate': '2025-08-20 20:00',
  'itemunit': 'μg/m3'},
 {'site': '大城',
  'county': '彰化縣',
  'pm25': '4',
  'datacreationdate': '2025-08-20 20:00',
  'itemunit': 'μg/m3'},
 {'site': '富貴角',
  'county': '新北市',
  'pm25': '3',
  'datacreationdate': '2025-08-20 20:00',
  'itemunit': 'μg/m3'},
 {'site': '麥寮',
  'county': '雲林縣',
  'pm25': '3',
  'datacreationdate': '2025-08-20 20:00',
  'itemunit': 'μg/m3'},
 {'site': '關山',
  'county': '臺東縣',
  'pm25': '10',
  'datacreationdate': '2025-08-20 20:00',
  'itemunit': 'μg/m3'},
 {'site': '馬公',
  'county': '澎湖縣',
  'pm25': '3',
  'datacreationdate': '2025-08-20 20:00',
  'itemunit': 'μg/m3'},
 {'site': '金門',
  'county': '金門縣',
  'pm25': '',
  'datacreationdate': '2025-08-20 20:00',
  'itemunit': 'μg/m3'},
 {'site': '馬祖',
  'county': '連江縣',
  'pm25': '17',
  'datacreationdate': '2025-08-20 20:00',
  'itemunit': 'μg/m3'},
 {'site': '埔里',
  'county': '南投縣',
  'pm25': '2',
  'datacreationdate'

In [8]:
len(datas)

1000

In [9]:
datas[0]

{'site': '員林',
 'county': '彰化縣',
 'pm25': '5',
 'datacreationdate': '2025-08-20 20:00',
 'itemunit': 'μg/m3'}

### 建立資料表

In [31]:
table_str="""
create table if not exists pm25(
id int auto_increment primary key,
site varchar(25),
county varchar(50),
pm25 int,
datacreationdate datetime,
itemunit varchar(20),
unique key site_time (site,datacreationdate)
)
"""
print(table_str)


create table if not exists pm25(
id int auto_increment primary key,
site varchar(25),
county varchar(50),
pm25 int,
datacreationdate datetime,
itemunit varchar(20),
unique key site_time (site,datacreationdate)
)



### 連接資料庫

In [32]:
conn=pymysql.connect(
host="localhost",
user="root",
password="",
port=3306,
database="demo"
)

cursor=conn.cursor()
cursor

In [12]:
cursor.execute(table_str)
conn.commit()

### 寫入一筆資料

In [13]:
sqlstr="insert into pm25(site,county,pm25,datacreationdate,itemunit)\
      values('{}','{}','{}','{}','{}')"
data=list(datas[0].values())
print(data)
sqlstr.format(data[0],data[1],data[2],data[3],data[4])

['員林', '彰化縣', '5', '2025-08-20 20:00', 'μg/m3']


"insert into pm25(site,county,pm25,datacreationdate,itemunit)      values('員林','彰化縣','5','2025-08-20 20:00','μg/m3')"

In [14]:
cursor.execute(sqlstr.format(data[0],data[1],data[2],data[3],data[4]))
conn.commit()

### 一次寫入
- 加上 ignore

In [33]:
sqlstr="insert ignore into pm25(site,county,pm25,datacreationdate,itemunit)\
      values(%s,%s,%s,%s,%s)"

# 移除pm25空字串部分
values=[list(data.values()) for data in datas if list(data.values())[2]!=""]

cursor.executemany(sqlstr,values)
conn.commit()

In [23]:
values

[['員林', '彰化縣', '5', '2025-08-20 20:00', 'μg/m3'],
 ['大城', '彰化縣', '4', '2025-08-20 20:00', 'μg/m3'],
 ['富貴角', '新北市', '3', '2025-08-20 20:00', 'μg/m3'],
 ['麥寮', '雲林縣', '3', '2025-08-20 20:00', 'μg/m3'],
 ['關山', '臺東縣', '10', '2025-08-20 20:00', 'μg/m3'],
 ['馬公', '澎湖縣', '3', '2025-08-20 20:00', 'μg/m3'],
 ['馬祖', '連江縣', '17', '2025-08-20 20:00', 'μg/m3'],
 ['埔里', '南投縣', '2', '2025-08-20 20:00', 'μg/m3'],
 ['復興', '高雄市', '5', '2025-08-20 20:00', 'μg/m3'],
 ['永和', '新北市', '9', '2025-08-20 20:00', 'μg/m3'],
 ['竹山', '南投縣', '0', '2025-08-20 20:00', 'μg/m3'],
 ['中壢', '桃園市', '7', '2025-08-20 20:00', 'μg/m3'],
 ['三重', '新北市', '11', '2025-08-20 20:00', 'μg/m3'],
 ['冬山', '宜蘭縣', '7', '2025-08-20 20:00', 'μg/m3'],
 ['宜蘭', '宜蘭縣', '6', '2025-08-20 20:00', 'μg/m3'],
 ['陽明', '臺北市', '4', '2025-08-20 20:00', 'μg/m3'],
 ['花蓮', '花蓮縣', '6', '2025-08-20 20:00', 'μg/m3'],
 ['臺東', '臺東縣', '8', '2025-08-20 20:00', 'μg/m3'],
 ['恆春', '屏東縣', '6', '2025-08-20 20:00', 'μg/m3'],
 ['潮州', '屏東縣', '7', '2025-08-20 20:00', 'μg/m3

In [35]:
conn.close()

In [29]:
import pandas as pd

In [43]:
cursor.execute("select * from pm25")

datas=cursor.fetchall()
datas

InterfaceError: (0, '')

In [36]:
df=pd.DataFrame(datas,columns=["id","site","county","pm25","datetime","unit"])
df

,id,site,county,pm25,datetime,unit
0,NaN,員林,彰化縣,5,NaN,NaN
1,NaN,大城,彰化縣,4,NaN,NaN
2,NaN,富貴角,新北市,3,NaN,NaN
3,NaN,麥寮,雲林縣,3,NaN,NaN
4,NaN,關山,臺東縣,10,NaN,NaN
...,...,...,...,...,...,...
995,NaN,大園,桃園市,7,NaN,NaN
996,NaN,桃園,桃園市,2,NaN,NaN
997,NaN,大同,臺北市,15,NaN,NaN
998,NaN,松山,臺北市,15,NaN,NaN


In [38]:
df[["site","datetime"]].drop_duplicates()

,site,datetime
0,員林,NaN
1,大城,NaN
2,富貴角,NaN
3,麥寮,NaN
4,關山,NaN
...,...,...
73,板橋,NaN
74,土城,NaN
75,新店,NaN
76,汐止,NaN
